In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [2]:

import sys
sys.path.append('../utils')
import functions

In [3]:
df = pd.read_csv('../data/local/raw/doesthedogdie.com.csv')
df.head()

,category_name,title,yes,no,yes_ratio
0,a dog dies,Movie_John Wick,1150,106,0.915605
1,a dog dies,Movie_Marley & Me,781,37,0.954768
2,a dog dies,Movie_Hereditary,680,67,0.910308
3,a dog dies,Movie_MEGAN,580,22,0.963455
4,a dog dies,TV Show_Game of Thrones,524,36,0.935714


In [4]:
functions.show_basic_info(df)


DataFrame Shape: (5910, 5)
Number of Rows: 5910
Number of Columns: 5

Data Types of Columns:
category_name     object
title             object
yes                int64
no                 int64
yes_ratio        float64
dtype: object

Missing Values per Column:
category_name    0
title            0
yes              0
no               0
yes_ratio        0
dtype: int64

First 5 Rows of Data:
  category_name                    title   yes   no  yes_ratio
0    a dog dies          Movie_John Wick  1150  106   0.915605
1    a dog dies        Movie_Marley & Me   781   37   0.954768
2    a dog dies         Movie_Hereditary   680   67   0.910308
3    a dog dies              Movie_MEGAN   580   22   0.963455
4    a dog dies  TV Show_Game of Thrones   524   36   0.935714


Round to 3 decimals in 'yes_ratio'

In [5]:
df['yes_ratio'] = df['yes_ratio'].round(3)

Check if there are duplicate titles

In [6]:
duplicates = df[df['title'].duplicated()]

duplicate_count = df['title'].duplicated().sum()

print(f"Number of duplicates in 'title': {duplicate_count}")

Number of duplicates in 'title': 5176


### Check the categories in 'title' column

In [7]:
# Extract the unique values before the underscore in 'title'
unique_titles = df['title'].str.split('_').str[0].unique()

print(unique_titles)

['Movie' 'TV Show' 'Video Game' 'Anime' 'Book' 'YouTube' 'Podcast' 'Manga'
 'Blog' 'Short Story']


Extract media type and remove it from title column

In [8]:
df['media_type'] = df['title'].str.split('_').str[0]  # Extract media type
df['title'] = df['title'].str.split('_').str[1]       # Remove media type from title

df.head()

,category_name,title,yes,no,yes_ratio,media_type
0,a dog dies,John Wick,1150,106,0.916,Movie
1,a dog dies,Marley & Me,781,37,0.955,Movie
2,a dog dies,Hereditary,680,67,0.910,Movie
3,a dog dies,MEGAN,580,22,0.963,Movie
4,a dog dies,Game of Thrones,524,36,0.936,TV Show


In [9]:
print(df['media_type'].unique())

['Movie' 'TV Show' 'Video Game' 'Anime' 'Book' 'YouTube' 'Podcast' 'Manga'
 'Blog' 'Short Story']


#### Create new df with media_type 'Movie' only

In [10]:
doesthe_movies = df[df['media_type'] == 'Movie']
print(doesthe_movies)

             category_name                              title   yes   no  \
0               a dog dies                          John Wick  1150  106   
1               a dog dies                        Marley & Me   781   37   
2               a dog dies                         Hereditary   680   67   
3               a dog dies                              MEGAN   580   22   
5               a dog dies                       The Babadook   514   15   
...                    ...                                ...   ...  ...   
5898  there's gun violence                              Joker   104    2   
5901  there's gun violence                    American Psycho   101    0   
5906  there's gun violence  Everything Everywhere All at Once    92    2   
5907  there's gun violence                        Jojo Rabbit    92    0   
5909  there's gun violence                                Saw    91    1   

      yes_ratio media_type  
0         0.916      Movie  
1         0.955      Movie  


In [11]:
doesthe_movies.to_csv('../data/local/raw/doesthedog_movies.csv', index=False)

In [12]:
print(f'Unique titles in doesthe_movies: {doesthe_movies['title'].nunique()}')

Unique titles in doesthe_movies: 433


In [13]:
doesthe_movies['clean_title'] = functions.prepare_clean_titles(doesthe_movies, 'title')
doesthe_movies.head()


/tmp/ipykernel_40683/2012065974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doesthe_movies['clean_title'] = functions.prepare_clean_titles(doesthe_movies, 'title')


,category_name,title,yes,no,yes_ratio,media_type,clean_title
0,a dog dies,John Wick,1150,106,0.916,Movie,john wick
1,a dog dies,Marley & Me,781,37,0.955,Movie,marley me
2,a dog dies,Hereditary,680,67,0.910,Movie,hereditary
3,a dog dies,MEGAN,580,22,0.963,Movie,megan
5,a dog dies,The Babadook,514,15,0.972,Movie,the babadook


In [14]:
functions.show_basic_info(doesthe_movies)


DataFrame Shape: (2836, 7)
Number of Rows: 2836
Number of Columns: 7

Data Types of Columns:
category_name     object
title             object
yes                int64
no                 int64
yes_ratio        float64
media_type        object
clean_title       object
dtype: object

Missing Values per Column:
category_name    0
title            0
yes              0
no               0
yes_ratio        0
media_type       0
clean_title      0
dtype: int64

First 5 Rows of Data:
  category_name         title   yes   no  yes_ratio media_type   clean_title
0    a dog dies     John Wick  1150  106      0.916      Movie     john wick
1    a dog dies   Marley & Me   781   37      0.955      Movie    marley  me
2    a dog dies    Hereditary   680   67      0.910      Movie    hereditary
3    a dog dies         MEGAN   580   22      0.963      Movie         megan
5    a dog dies  The Babadook   514   15      0.972      Movie  the babadook


Drop media_type column

In [15]:
doesthe_movies.drop(columns=['media_type'], inplace=True)

/tmp/ipykernel_40683/2280681998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doesthe_movies.drop(columns=['media_type'], inplace=True)


#### Extract category_name unique values

In [16]:
unique_values = sorted(doesthe_movies['category_name'].dropna().unique())
print(f'Unique values in category column: {unique_values}')

Unique values in category column: ["A child's dear toy is destroyed", 'Autism specific abuse', 'D.I.D. Misrepresentation', 'Santa (et al) is spoiled', 'Someone attempts suicide', 'Someone becomes unconscious', "There's audio gore", 'a baby cries', 'a baby is stillborn', 'a car crashes', 'a car honks or tires screech', 'a cat dies', 'a child is abandoned by a parent or guardian', 'a dog dies', 'a dragon dies', 'a family member dies', 'a horse dies', 'a kid dies', 'a major character dies', 'a male character is ridiculed for crying', 'a mentally ill person is violent', 'a minor is sexualized', 'a minority is misrepresented', 'a non-human character dies', 'a parent dies', 'a person is hit by a car', 'a pet dies', 'a plane crashes', 'a pregnant person dies', 'a priceless artifact is destroyed', 'a trans person is depicted predatorily', 'a woman gets slapped', 'a woman is brutalized for spectacle', 'alcohol abuse', 'an LGBT person dies', 'an LGBT+ person is outed', 'an animal dies', 'an anim

In [17]:
print(f'Unique titles in doesthe_movies: {doesthe_movies['title'].nunique()}')

Unique titles in doesthe_movies: 433


### Drop rows where 'yes' is greater than 'no'
Keep rows where sensitive content does take place.

In [18]:
events_df = doesthe_movies.copy()
events_df = events_df[events_df['yes'] >= events_df['no']]

Group categories in new column

In [19]:
events_df['events'] = events_df.groupby('clean_title')['category_name'].transform(lambda x: ', '.join(x))
events_df = events_df.drop(columns=['category_name'])

In [23]:
events_df.head(10)

,title,yes,no,yes_ratio,clean_title,events
0,John Wick,1150,106,0.916,john wick,"a dog dies, there's a dead animal, a pet dies,..."
1,Marley & Me,781,37,0.955,marley me,"a dog dies, a pet dies, an animal is sad, some..."
2,Hereditary,680,67,0.910,hereditary,"a dog dies, there's a dead animal, there are b..."
3,MEGAN,580,22,0.963,megan,"a dog dies, there's a dead animal, there are b..."
5,The Babadook,514,15,0.972,the babadook,"a dog dies, there are bugs, a pet dies, animal..."
6,A Dog's Purpose,440,20,0.957,a dogs purpose,"a dog dies, a cat dies"
7,I Am Legend,434,13,0.971,i am legend,"a dog dies, there's dog fighting"
8,The Conjuring,434,14,0.969,the conjuring,"a dog dies, a pet dies, an animal dies, there ..."
10,American Psycho,378,16,0.959,american psycho,"a dog dies, there's a dead animal, a pet dies,..."
13,The Witch,301,8,0.974,the witch,"a dog dies, a kid dies, an infant is abducted"


In [21]:
events_df

,title,yes,no,yes_ratio,clean_title,events
0,John Wick,1150,106,0.916,john wick,"a dog dies, there's a dead animal, a pet dies,..."
1,Marley & Me,781,37,0.955,marley me,"a dog dies, a pet dies, an animal is sad, some..."
2,Hereditary,680,67,0.910,hereditary,"a dog dies, there's a dead animal, there are b..."
3,MEGAN,580,22,0.963,megan,"a dog dies, there's a dead animal, there are b..."
5,The Babadook,514,15,0.972,the babadook,"a dog dies, there are bugs, a pet dies, animal..."
...,...,...,...,...,...,...
5898,Joker,104,2,0.981,joker,"someone is stalked, heads get squashed, there ..."
5901,American Psycho,101,0,1.000,american psycho,"a dog dies, there's a dead animal, a pet dies,..."
5906,Everything Everywhere All at Once,92,2,0.979,everything everywhere all at once,"animals are abused, a child is abandoned by a ..."
5907,Jojo Rabbit,92,0,1.000,jojo rabbit,"rabbits are harmed, there's fat jokes, there's..."
